In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

from catboost import CatBoostRegressor, Pool, cv
from catboost import MetricVisualizer

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from scipy.stats import boxcox
from os import listdir

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import shap
shap.initjs()

In [ ]:
print(listdir("../input"))

In [ ]:
data = pd.read_csv("../input/ecommerce-data/data.csv", encoding="ISO-8859-1", dtype={'CustomerID': str})
data.shape

ASCII: 7 bits. 128 code points.

ISO-8859-1: 8 bits. 256 code points.

UTF-8: 8-32 bits (1-4 bytes). 1,112,064 code points.

In [ ]:
data.head()

We can see that the datafile has information given for each single transaction. Take a look at the InvoiceNo and the CustomerID of the first entries. Here we can see that one customer with ID 17850 of the United Kingdom made a single order that has the InvoideNo 536365. The customer ordered several products with different stockcodes, descriptions, unit prices and quantities. In addition we can see that the InvoiceDate was the same for these products.

In [ ]:
data.isnull().sum()

In [ ]:
missing_percentage = data.isnull().sum() / data.shape[0] * 100
missing_percentage

In [ ]:
data[data["Description"].isnull()].head()

How often do we miss the customer as well?

In [ ]:
data[data["Description"].isnull()].CustomerID.isnull().value_counts()

In [ ]:
data[data["Description"].isnull()].InvoiceDate.value_counts() #checking other attributes

In [ ]:
data[data["Description"].isnull()].StockCode.value_counts() #etc.

In [ ]:
data[data["Description"].isnull()].UnitPrice.value_counts()

In cases of missing descriptions we always miss the customer and the unit price as well. Why does the retailer records such kind of entries without a further description? It seems that there is no sophisticated procedure how to deal with and record such kind of transactions. This is already a hint that we could expect strange entries in our data and that it can be difficult to detect them!

In [ ]:
data.CustomerID.isnull().sum()

In [ ]:
data[data.CustomerID.isnull()].head()

In [ ]:
data.loc[data.CustomerID.isnull(), ["UnitPrice", "Quantity"]].describe()

That's bad as well. The price and the quantities of entries without a customer ID can show extreme outliers. As we might want to create features later on that are based on historical prices and sold quantities, this is very disruptive. Our first advice for the retailer is to setup strategies for transactions that are somehow faulty or special. And the question remains: Why is it possible for a transaction to be without a customer ID. Perhaps you can purchase as a quest but then it would of a good and clean style to plugin a special ID that indicates that this one is a guest. Ok, next one: Do we have hidden nan-values in Descriptions? To find it out, let's create a new feature that hold descriptions in lowercase:

Hidden missing descriptions

Can we find "nan"-Strings?

In [ ]:
data.loc[data.Description.isnull()==False, "lowercase_descriptions"] = data.loc[
data.Description.isnull()==False,"Description"
].apply(lambda l: l.lower())

data.lowercase_descriptions.dropna().apply(
    lambda l: np.where("nan" in l, True, False)
).value_counts()

 can we find empty ""-strings?

In [ ]:
data.lowercase_descriptions.dropna().apply(
    lambda l: np.where("" == l, True, False)
).value_counts()

We found additional, hidden nan-values that show a string "nan" instead of a nan-value. Let's transform them to NaN:

In [ ]:
data.loc[data.lowercase_descriptions.isnull()==False, "lowercase_descriptions"] = data.loc[
    data.lowercase_descriptions.isnull()==False, "lowercase_descriptions"
].apply(lambda l: np.where("nan" in l, None, l))

In [ ]:
data.head()

In [ ]:
#data.iloc[622,:]

As we don't know why customers or descriptions are missing and we have seen strange outliers in quantities and prices as well as zero-prices, let's play safe and drop all of these occurences.

In [ ]:
data = data.loc[(data.CustomerID.isnull()==False) & (data.lowercase_descriptions.isnull()==False)].copy()

In [ ]:
data.isnull().sum()

In [ ]:
data.isnull().sum().sum()

The Time period 

How long is the period in days?

In [ ]:
data["InvoiceDate"] = pd.to_datetime(data.InvoiceDate, cache=True)

data.InvoiceDate.max() - data.InvoiceDate.min()

In [ ]:
print("Datafile starts with timepoint {}".format(data.InvoiceDate.min()))
print("Datafile ends with timepoint {}".format(data.InvoiceDate.max()))

The invoice number 

How many different invoice numbers do we have?

In [ ]:
data.InvoiceNo.nunique()

In the data description we can find that a cancelled transactions starts with a "C" in front of it. Let's create a feature to easily filter out these cases:

In [ ]:
x = pd.DataFrame(np.where(data.InvoiceNo.apply(lambda l: l[0]=="C")))
x

In [ ]:
data.iloc[141,:]

In [ ]:
data["IsCancelled"]=np.where(data.InvoiceNo.apply(lambda l: l[0]=="C"), True, False)
data.IsCancelled.value_counts() / data.shape[0] * 100

2.2 % of all entries are cancellations.

In [ ]:
data.loc[data.IsCancelled==True].describe()

All cancellations have negative quantites but positive, non-zero unit prices. Given this data we are not easily able to understand why a customer made a return and it's very difficult to predict such cases as there could be several, hidden reasons why a cancellation was done. Let's drop them:

In [ ]:
data = data.loc[data.IsCancelled==False].copy()
data = data.drop("IsCancelled", axis=1)

Stockcodes 

How many unique stockcodes do we have?

In [ ]:
data.StockCode.nunique()

Which codes are most common?

In [ ]:
stockcode_counts = data.StockCode.value_counts().sort_values(ascending=False)
fig, ax = plt.subplots(2,1,figsize=(20,15))
sns.barplot(stockcode_counts.iloc[0:20].index,
            stockcode_counts.iloc[0:20].values,
            ax = ax[0], palette="Oranges_r")
ax[0].set_ylabel("Counts")
ax[0].set_xlabel("Stockcode")
ax[0].set_title("Which stockcodes are most common?");
sns.distplot(np.round(stockcode_counts/data.shape[0]*100,2),
             #kde=False,
             bins=20,
             ax=ax[1], color="Orange")
ax[1].set_title("How seldom are stockcodes?")
ax[1].set_xlabel("% of data with this stockcode")
ax[1].set_ylabel("Frequency");

Do you see the POST in the most common stockcode counts?! That's a strange one! Hence we could expect strange occurences not only in the descriptions and customerIDs but also in the stockcode. OHOHOH! It's code is shorter than the others as well as not numeric.

Most stockcodes are very seldom. This indicates that the retailer sells many different products and that there is no strong specialization of a specific stockcode. Nevertheless we have to be careful as this must not mean that the retailer is not specialized given a specific product type. The stockcode could be a very detailed indicator that does not yield information of the type, for example water bottles may have very different variants in color, name and shapes but they are all water bottles.

Let's count the number of numeric chars in and the length of the stockcode:

In [ ]:
def count_numeric_chars(l):
    return sum(1 for c in l if c.isdigit())

data["StockCodeLength"] = data.StockCode.apply(lambda l: len(l))
data["nNumericStockCode"] = data.StockCode.apply(lambda l: count_numeric_chars(l))

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,5))
sns.countplot(data["StockCodeLength"], palette="Oranges_r", ax=ax[0])
sns.countplot(data["nNumericStockCode"], palette="Oranges_r", ax=ax[1])
ax[0].set_xlabel("Length of stockcode")
ax[1].set_xlabel("Number of numeric chars in the stockcode");

Even though the majority of samples has a stockcode that consists of 5 numeric chars, we can see that there are other occurences as well. The length can vary between 1 and 12 and there are stockcodes with no numeric chars at all!

In [ ]:
data.loc[data.nNumericStockCode < 5]

In [ ]:
data.loc[data.nNumericStockCode < 5].lowercase_descriptions.value_counts()

again something that we don't want to predict. Again this indicates that the retailer does not speparate well between special kind of transactions and valid customer-retailer transactions. Let's drop all of these occurences:

In [ ]:
data = data.loc[(data.nNumericStockCode == 5) & (data.StockCodeLength==5)].copy()
data.StockCode.nunique()

In [ ]:
data = data.drop(["nNumericStockCode", "StockCodeLength"], axis=1)

In [ ]:
data

Descriptions 

How many unique descriptions do we have?

In [ ]:
data.Description.nunique()

And which are most common?

In [ ]:
description_counts = data.Description.value_counts().sort_values(ascending=False).iloc[0:50]
plt.figure(figsize=(20,5))
sns.barplot(description_counts.index, description_counts.values, palette="Purples_r")
plt.ylabel("Counts")
plt.title("Which product descriptions are most common?");
plt.xticks(rotation=90);

Ok, we can see that some descriptions correspond to a similar product type. Do you see the multiple occurences of lunch bags? We often have color information about the product as well. Furthermore the most common descriptions seem to confirm that the retailer sells various different kinds of products. All descriptions seem to consist of uppercase chars. Ok, now let's do some addtional analysis on the descriptions by counting the length and the number of lowercase chars.

In [ ]:
def count_lower_chars(l):
    return sum(1 for c in l if c.islower())

In [ ]:
data["DescriptionLength"] = data.Description.apply(lambda l: len(l))
data["LowCharsInDescription"] = data.Description.apply(lambda l: count_lower_chars(l))

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,5))
sns.countplot(data.DescriptionLength, ax=ax[0], color="Violet")
sns.countplot(data.LowCharsInDescription, ax=ax[1], color="Purple")
ax[1].set_yscale("log")

Oh, great! Almost all descriptions do not have a lowercase chars, but we have found exceptional cases!

In [ ]:
lowchar_counts = data.loc[data.LowCharsInDescription > 0].Description.value_counts()

plt.figure(figsize=(15,3))
sns.barplot(lowchar_counts.index, lowchar_counts.values, palette="Purples_r")
plt.xticks(rotation=90);

Next day carriage and high resolution image are strange! Let's compute the fraction of lower with respect to uppercase letters:

In [ ]:
def count_upper_chars(l):
    return sum(1 for c in l if c.isupper())

data["UpCharsInDescription"] = data.Description.apply(lambda l: count_upper_chars(l))

In [ ]:
data.UpCharsInDescription.describe()

In [ ]:
data.loc[data.UpCharsInDescription <=5].Description.value_counts()

It's strange that they differ from the others. Let's drop them:

In [ ]:
data = data.loc[data.UpCharsInDescription > 5].copy()

And what about the descriptions with a length below 14?

In [ ]:
dlength_counts = data.loc[data.DescriptionLength < 14].Description.value_counts()

plt.figure(figsize=(20,5))
sns.barplot(dlength_counts.index, dlength_counts.values, palette="Purples_r")
plt.xticks(rotation=90);

Ok, descriptions with small length look valid and we should not drop them. Ok, now let's see how many unique stock codes do we have and how many unique descriptions?

In [ ]:
data.StockCode.nunique()

In [ ]:
data.Description.nunique()

We still have more descriptions than stockcodes and we should continue to find out why they differ.

In [ ]:
data.groupby("StockCode").Description.nunique().sort_values(ascending=False).iloc[0:10]

Whoa, we still have stockcodes with multiple descriptions. Let's look at an example:

In [ ]:
data.loc[data.StockCode == "23244"].Description.value_counts()

In [ ]:
data.loc[data.StockCode == "23196"].Description.value_counts()

Ok, browsing through the cases we can see that stockcodes are sometimes named a bit differently due to missing or changed words or typing errors. Nonetheless they look ok and we can continue.

Customers

In [ ]:
data.CustomerID.nunique()

In [ ]:
data.CustomerID.unique()

In [ ]:
customer_counts = data.CustomerID.value_counts().sort_values(ascending=False).iloc[0:20] 
plt.figure(figsize=(20,5))
sns.barplot(customer_counts.index, customer_counts.values, order=customer_counts.index)
plt.ylabel("Counts")
plt.xlabel("CustomerID")
plt.title("Which customers are most common?");
#plt.xticks(rotation=90);

In [ ]:
"""
description_counts = data.Description.value_counts().sort_values(ascending=False).iloc[0:50]
plt.figure(figsize=(20,5))
sns.barplot(description_counts.index, description_counts.values, palette="Purples_r")
plt.ylabel("Counts")
plt.title("Which product descriptions are most common?");
plt.xticks(rotation=90);
"""

Countries 

How many unique countries are delivered by the retailer?

In [ ]:
data.Country.nunique()

And which ones are most common?

In [ ]:
Country_counts = data.Country.value_counts().sort_values(ascending=False).iloc[0:20] 
plt.figure(figsize=(20,5))
sns.barplot(Country_counts.index, customer_counts.values, palette="Greens_r")
#plt.yscale("log")
plt.ylabel("Counts")
plt.xlabel("Country")
plt.title("Which countries made the most transactions?");
plt.xticks(rotation=45);

We can see that the retailer sells almost all products in the UK, followed by many european countries. How many percentage of entries are inside UK?

In [ ]:
data.loc[data.Country=="United Kingdom"].shape

In [ ]:
data.loc[data.Country=="United Kingdom"].shape[0] / data.shape[0] * 100

Unit Price

In [ ]:
data.UnitPrice.describe()

Again, we have strange occurences: zero unit prices!

In [ ]:
data.loc[data.UnitPrice == 0].sort_values(by="Quantity", ascending=False).head()

That's not good again. It's not obvious if they are gifts to customers or not :-( Let's drop them:

In [ ]:
data = data.loc[data.UnitPrice > 0].copy()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,5))
sns.distplot(data.UnitPrice, ax=ax[0], kde=False, color="red")
sns.distplot(np.log(data.UnitPrice), ax=ax[1], bins=20, color="tomato", kde=False)
ax[1].set_xlabel("Log-Unit-Price");

In [ ]:
np.exp(-2)

In [ ]:
np.exp(3)

In [ ]:
np.quantile(data.UnitPrice, 0.95)

Let's focus transactions with prices that fall into this range as we don't want to make predictions for very seldom products with high prices. Starting easy is always good!

In [ ]:
data = data.loc[(data.UnitPrice > 0.1) & (data.UnitPrice < 20)].copy()

Quantities 

Ok, the most important one - the target. Let's take a look at its distribution:

In [ ]:
data.Quantity.describe()

Ok, most products are sold in quantities from 1 to 12. But, we have extreme, unrealistic outliers again:

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,5))
sns.distplot(data.Quantity, ax=ax[0], kde=False, color="limegreen");
sns.distplot(np.log(data.Quantity), ax=ax[1], bins=20, kde=False, color="limegreen");
ax[0].set_title("Quantity distribution")
ax[0].set_yscale("log")
ax[1].set_title("Log-Quantity distribution")
ax[1].set_xlabel("Natural-Log Quantity");

As you can see by the log-transformed distribution it would make sense to make a cut at:

In [ ]:
np.exp(4)

In [ ]:
np.quantile(data.Quantity, 0.95)

In [ ]:
data = data.loc[data.Quantity < 55].copy()

How to predict daily product sales? 
In this kernel I like to use catboost as predictive model. The prediction of daily quantities and revenues are both regression tasks and consequently I will use the catboost regressor. The loss and metric I like to use is the root mean square error (RMSE):

# Another EDA solution


Context of Data
Company - UK-based and registered non-store online retail

Products for selling - Mainly all-occasion gifts

Customers - Most are wholesalers (local or international)

Transactions Period - 1st Dec 2010 - 9th Dec 2011 (One year)

**Results obtained from Exploratory Data Analysis (EDA)**
The customer with the highest number of orders comes from the United Kingdom (UK) customer with the highest money spent on purchases comes from Netherlands.The company receives the highest number of orders from customers in the UK (since it is a UK-based company). Therefore, the TOP 5 countries (including UK) that place the highest number of orders are as below:

1. United Kingdom
1. Germany
1. France
1. Ireland (EIRE)
1. Spain

As the company receives the highest number of orders from customers in the UK (since it is a UK-based company), customers in the UK spend the most on their purchases. Therefore, the TOP 5 countries (including UK) that spend the most money on purchases are as below:

1. United Kingdom
1. Netherlands
1. Ireland (EIRE)
1. Germany
1. France

November 2011 has the highest sales

The month with the lowest sales is undetermined as the dataset consists of transactions until 9th December 2011 in December
There are no transactions on Saturday between 1st Dec 2010 - 9th Dec 2011

The number of orders received by the company tends to increases from Monday to Thursday and decrese afterward

The company receives the highest number of orders at 12:00pm

Possibly most customers made purchases during lunch hour between 12:00pm - 2:00pm

The company tends to give out FREE items for purchases occasionally each month (Except June 2011)

However, it is not clear what factors contribute to giving out the FREE items to the particular customers

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

import warnings
# current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

import missingno as msno # missing data visualization module for Python
import pandas_profiling

import gc
import datetime

%matplotlib inline
color = sns.color_palette()

In [ ]:
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 100)

In [ ]:
# specify encoding to deal with different formats
df = pd.read_csv('../input/ecommerce-data/data.csv', encoding = 'ISO-8859-1')
df.head()

In [ ]:
customer_country=data[['Country','CustomerID']].drop_duplicates()
customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID', ascending=False)

So, it indicates that More than 90% of the data is coming from UK !

In [ ]:
# change the column names
df.rename(index=str, columns={'InvoiceNo': 'invoice_num',
                              'StockCode' : 'stock_code',
                              'Description' : 'description',
                              'Quantity' : 'quantity',
                              'InvoiceDate' : 'invoice_date',
                              'UnitPrice' : 'unit_price',
                              'CustomerID' : 'cust_id',
                              'Country' : 'country'}, inplace=True)
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# check missing values for each column 

df.isnull().sum(axis=0).sort_values() #ascending=False


In [ ]:
#Let's take a look at the missing values

plt.figure(figsize=(5, 5))
df.isnull().mean(axis=0).plot.barh()
plt.title("Ratio of missing values per columns")

In [ ]:
#check the dupplicate values and drop them if there's any.
"""
print('Dupplicate entries: {}'.format(data.duplicated().sum()))
data.drop_duplicates(inplace = True)

"""

In [ ]:
# check out the rows with missing values

df[df.isnull().any(axis=1)].head()

In [ ]:
df.isnull().sum(axis=1) #“axis 0” represents rows and “axis 1” represents columns

In [ ]:
df.country.nunique()

In [ ]:
# change the invoice_date format - String to Timestamp format

df['invoice_date'] = pd.to_datetime(df.invoice_date, format='%m/%d/%Y %H:%M')

In [ ]:
# change description - UPPER case to LOWER case

df['description'] = df.description.str.lower()

In [ ]:
df.head()

In [ ]:
# df_new without missing values
df_new = df.dropna()

In [ ]:
# check missing values for each column 
df_new.isnull().sum().sort_values(ascending=False)

In [ ]:
df_new.info()

In [ ]:
# change columns tyoe - String to Int type 

df_new['cust_id'] = df_new['cust_id'].astype('int64')
df.head()

In [ ]:
df_new.describe().round(2)

Remove Quantity with negative values

In [ ]:
df_new = df_new[df_new.quantity > 0]
df_new.describe().round(2)

Add the column - amount_spent

In [ ]:
df_new['amount_spent'] = df_new['quantity'] * df_new['unit_price']

# rearrange all the columns for easy reference
df_new = df_new[['invoice_num','invoice_date','stock_code','description','quantity',
                 'unit_price','amount_spent','cust_id','country']]

Add the columns - Month, Day and Hour for the invoice

In [ ]:
df_new.insert(loc=2, column='year_month', value=df_new['invoice_date'].map(lambda x: 100*x.year + x.month))

df_new.insert(loc=3, column='month', value=df_new.invoice_date.dt.month)

# +1 to make Monday=1.....until Sunday=7

df_new.insert(loc=4, column='day', value=(df_new.invoice_date.dt.dayofweek)+1)

df_new.insert(loc=5, column='hour', value=df_new.invoice_date.dt.hour)

df_new.head()

# Exploratory Data Analysis (EDA)

How many orders made by the customers?


In [ ]:
df_new.groupby(by=['cust_id','country'], as_index=False)['invoice_num'].count().head(10)

In [ ]:
orders = df_new.groupby(by=['cust_id','country'], as_index=False)['invoice_num'].count()

plt.subplots(figsize=(15,6))
plt.plot(orders.cust_id, orders.invoice_num)
plt.xlabel('Customers ID')
plt.ylabel('Number of Orders')
plt.title('Number of Orders for different Customers')
plt.show()

Check TOP 5 most number of orders

In [ ]:
print('The TOP 5 customers with most number of orders...')
orders.sort_values(by = "invoice_num", ascending = False).iloc[0:5]
#orders_Sorted_by_invoiceNo.head()

In [ ]:
orders_Sorted_by_invoiceNo = orders.sort_values(by = "invoice_num", ascending = False)

orders_Sorted_by_invoiceNo_country = orders_Sorted_by_invoiceNo.country.iloc[0:5]
orders_Sorted_by_invoiceNo_invoice_num = orders_Sorted_by_invoiceNo.invoice_num.iloc[0:5]

plt.subplots(figsize=(10,6))

barlist = plt.bar(orders_Sorted_by_invoiceNo_country, orders_Sorted_by_invoiceNo_invoice_num)
barlist[0].set_color('r')

#sns.barplot(x = orders_Sorted_by_invoiceNo_country, y = orders_Sorted_by_invoiceNo_invoice_num, data = df_new)
plt.xlabel('Country')
plt.ylabel('Number of Orders')
plt.title('Top 5 countries with most number of orders ')
plt.show()

How much money spent by the customers?

In [ ]:
df_new.groupby(by=['cust_id','country'], as_index=False)['amount_spent'].count().head()

In [ ]:
money_spent = df_new.groupby(by=['cust_id','country'], as_index=False)['amount_spent'].sum()

plt.subplots(figsize=(15,6))
plt.plot(money_spent.cust_id, money_spent.amount_spent)
plt.xlabel('Customers ID')
plt.ylabel('Number of Orders')
plt.title('Number of Orders for different Customers')
plt.show()

Check TOP 5 highest money spent

In [ ]:
print('The TOP 5 customers with highest money spent...')
money_spent.sort_values(by='amount_spent', ascending=False).head()

Discover patterns for Number of Orders

How many orders (per month)?

In [ ]:
# color available
sns.palplot(color)

In [ ]:
df_new.groupby('invoice_num')['year_month'].unique().value_counts()

In [ ]:
ax = df_new.groupby('invoice_num')['year_month'].unique().value_counts().sort_index().plot('bar',color=color[0],figsize=(15,6))
ax.set_xlabel('Month',fontsize=15)
ax.set_ylabel('Number of Orders',fontsize=15)
ax.set_title('Number of orders for different Months (1st Dec 2010 - 9th Dec 2011)',fontsize=15)
ax.set_xticklabels(('Dec_10','Jan_11','Feb_11','Mar_11','Apr_11','May_11','Jun_11','July_11','Aug_11','Sep_11','Oct_11','Nov_11','Dec_11'), rotation='horizontal', fontsize=13)
plt.show()

In [ ]:
df_new.groupby('invoice_num')['day'].unique().value_counts().sort_index()

In [ ]:
ax = df_new.groupby('invoice_num')['day'].unique().value_counts().sort_index().plot('bar',color=color[0],figsize=(15,6))
ax.set_xlabel('Day',fontsize=15)
ax.set_ylabel('Number of Orders',fontsize=15)
ax.set_title('Number of orders for different Days',fontsize=15)
ax.set_xticklabels(('Mon','Tue','Wed','Thur','Fri','Sun'), rotation='horizontal', fontsize=15)
plt.show()

How many orders (per hour)?

In [ ]:
df_new.groupby('invoice_num')['hour'].unique().value_counts().iloc[:-1].sort_index()

In [ ]:
ax = df_new.groupby('invoice_num')['hour'].unique().value_counts().iloc[:-1].sort_index().plot('bar',color=color[0],figsize=(15,6))
ax.set_xlabel('Hour',fontsize=15)
ax.set_ylabel('Number of Orders',fontsize=15)
ax.set_title('Number of orders for different Hours',fontsize=15)
ax.set_xticklabels(range(6,21), rotation='horizontal', fontsize=15)
plt.show()

Discover patterns for Unit Price

In [ ]:
df_new.unit_price.describe()

We see that there are unit price = 0 (FREE items)

There are some free items given to customers from time to time.

In [ ]:
# check the distribution of unit price
plt.subplots(figsize=(12,6))
sns.boxplot(df_new.unit_price)
plt.show()

In [ ]:
df_free = df_new[df_new.unit_price == 0]
df_free.head()

In [ ]:
df_free.shape

In [ ]:
df_free.year_month.value_counts().sort_index()

In [ ]:
ax = df_free.year_month.value_counts().sort_index().plot('bar',figsize=(12,6), color=color[0])
ax.set_xlabel('Month',fontsize=15)
ax.set_ylabel('Frequency',fontsize=15)
ax.set_title('Frequency for free gifts for different Months (Dec 2010 - Dec 2011)',fontsize=15)
ax.set_xticklabels(('Dec_10','Jan_11','Feb_11','Mar_11','Apr_11','May_11','July_11','Aug_11','Sep_11','Oct_11','Nov_11'), rotation='horizontal', fontsize=13)
plt.show()

Not clear why there are FREE items given to certain customers

On average, the company gave out 2-4 times FREE items to customers each month (Except in June 2011)

Discover patterns for each Country

In [ ]:
df_new.head()

How many orders for each country?

In [ ]:
df_new.groupby('country')['invoice_num'].count().sort_values(ascending = False)

In [ ]:
group_country_orders = df_new.groupby('country')['invoice_num'].count().sort_values()
# del group_country_orders['United Kingdom']

# plot number of unique customers in each country (with UK)
plt.subplots(figsize=(15,8))
group_country_orders.plot('barh', fontsize=12, color=color[0])
plt.xlabel('Number of Orders', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.title('Number of Orders for different Countries', fontsize=12)
plt.show()

In [ ]:
group_country_orders = df_new.groupby('country')['invoice_num'].count().sort_values()
del group_country_orders['United Kingdom']

# plot number of unique customers in each country (without UK)
plt.subplots(figsize=(15,8))
group_country_orders.plot('barh', fontsize=12, color=color[0])
plt.xlabel('Number of Orders', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.title('Number of Orders for different Countries', fontsize=12)
plt.show()

How much money spent by each country?

In [ ]:
group_country_amount_spent = df_new.groupby('country')['amount_spent'].sum().sort_values()
# del group_country_orders['United Kingdom']

# plot total money spent by each country (with UK)
plt.subplots(figsize=(15,8))
group_country_amount_spent.plot('barh', fontsize=12, color=color[0])
plt.xlabel('Money Spent (Dollar)', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.title('Money Spent by different Countries', fontsize=12)
plt.show()

In [ ]:
group_country_amount_spent = df_new.groupby('country')['amount_spent'].sum().sort_values()
del group_country_amount_spent['United Kingdom']

# plot total money spent by each country (without UK)
plt.subplots(figsize=(15,8))
group_country_amount_spent.plot('barh', fontsize=12, color=color[0])
plt.xlabel('Money Spent (Dollar)', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.title('Money Spent by different Countries', fontsize=12)
plt.show()

In [ ]:
df_new.head(2)

In [ ]:
# Constucting a basket for later use
temp = df_new.groupby(by=['cust_id', 'invoice_num'], as_index=False)['invoice_date'].count()
nb_products_per_basket = temp.rename(columns = {'InvoiceDate':'Number of products'})

In [ ]:
nb_products_per_basket.invoice_num = nb_products_per_basket.invoice_num.astype(str)
nb_products_per_basket['order_canceled'] = nb_products_per_basket['invoice_num'].apply(lambda x:int('C' in x))
len(nb_products_per_basket[nb_products_per_basket['order_canceled']==1])/len(nb_products_per_basket)*100

these lines are for checking cancelled transactions, in the notebook it was 16% , here it is 0.

Let's take a look at some rows where the transaction was canceled.

In [ ]:
nb_products_per_basket[nb_products_per_basket['order_canceled']==1][:5]

# Another notebook

RFM (Recency Frequency Monetary) Analysis
RFM is a method used for analyzing customer value. It is commonly used in database marketing and direct marketing and has received particular attention in retail and professional services industries

RFM stands for the three dimensions:

1. Recency – How recently did the customer purchase?
1. Frequency – How often do they purchase?
1. Monetary Value – How much do they spend?

Before starting RFM analysis we need to set objective and outcome of analysis, for this example goal is to define class of customer example - Platinum, Gold and Silver.

1. Platinum customer - frequent and more revenue generator.
1. Gold customer - frequent but less revenue generator.
1. silver customer - less frequent and less revenue generator.

In [ ]:
#importing all important package..

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("../input/ecommerce-data/data.csv", encoding="ISO-8859-1")
df.head()

In [ ]:
#information of dataset..
df.info()

In [ ]:
#Country with high count must be taken for testing purpose... can we divide based on demographic or similar taste

df.Country.value_counts().head(5)

df = df[df.Country == 'United Kingdom']

In [ ]:
df.head(10)

In [ ]:
#checking distribution of quantity..

sns.violinplot(df.Quantity)

In [ ]:
df.Quantity.describe()

In [ ]:
#Quantity can not be negative so remove negative values..
df = df[df['Quantity']>0]
df.Quantity.describe()

In [ ]:
#checking distribution of unit price..
sns.violinplot(df.UnitPrice)

In [ ]:
df = df[df['UnitPrice']>0]
df.UnitPrice.describe()

In [ ]:
#checking null values in all columns in dataset
null_values = pd.DataFrame(df.isnull().sum(),columns=['count_value'])
ax = sns.barplot(null_values.count_value,null_values.index)

In [ ]:
null_values.head(10)

In [ ]:
df.dropna(subset=['CustomerID'],how='all',inplace=True)
df.isnull().sum()

In [ ]:
#last date available in our dataset
df['InvoiceDate'].max()

In [ ]:
#use latest date in our data as current date..

import datetime as dt
now = dt.date(2011,12,9)

In [ ]:
df['date'] = pd.DatetimeIndex(df.InvoiceDate).date
df.head()

In [ ]:
#group by customer by last date they purchased...

recency_df = df.groupby(['CustomerID'],as_index=False)['date'].max()
recency_df.columns = ['CustomerID','LastPurchaseDate']
recency_df.head()

In [ ]:
#calculate how often he is purchasing with reference to latest date in days..

recency_df['Recency'] = recency_df.LastPurchaseDate.apply(lambda x : (now - x).days)
recency_df.head()

In [ ]:
recency_df.drop(columns=['LastPurchaseDate'],inplace=True)

In [ ]:
#check frequency of customer means how many transaction has been done..

frequency_df = df.copy()
frequency_df.drop_duplicates(subset=['CustomerID','InvoiceNo'], keep="first", inplace=True) 
frequency_df = frequency_df.groupby('CustomerID',as_index=False)['InvoiceNo'].count()
frequency_df.columns = ['CustomerID','Frequency']
frequency_df.head()

In [ ]:
#check frequency of customer means how many transaction has been done..

frequency_df = df.copy()
frequency_df.drop_duplicates(subset=['CustomerID','InvoiceNo'], keep="first", inplace=True) 

frequency_df = frequency_df.groupby('CustomerID',as_index=False)['InvoiceNo'].count()

frequency_df.columns = ['CustomerID','Frequency']
frequency_df.head()

In [ ]:
#calculate how much a customer spend in the each transaction...

df['Total_cost'] = df['UnitPrice'] * df['Quantity']

In [ ]:
#check summed up spend of a customer with respect to latest date..

monetary_df=df.groupby('CustomerID',as_index=False)['Total_cost'].sum()
monetary_df.columns = ['CustomerID','Monetary']

monetary_df.head()

In [ ]:
#Combine all together all dataframe in so we have recency, frequency and monetary values together..

#combine first recency and frequency..
rf = recency_df.merge(frequency_df,left_on='CustomerID',right_on='CustomerID')

#combibe rf frame with monetary values..

rfm = rf.merge(monetary_df,left_on='CustomerID',right_on='CustomerID')

rfm.set_index('CustomerID',inplace=True)

rfm.head()

In [ ]:
#checking correctness of output..

df[df.CustomerID == 12346.0]

In [ ]:
(now - dt.date(2011,1,18)).days == 325

In [ ]:
#bring all the quartile value in a single dataframe

rfm_segmentation = rfm.copy()

In [ ]:
from sklearn.cluster import KMeans

# get right number of cluster for K-means so we neeed to loop from 1 to 20 number of cluster and check score.
#Elbow method is used to represnt that. 

Nc = range(1, 20)
kmeans = [KMeans(n_clusters=i) for i in Nc]
score = [kmeans[i].fit(rfm_segmentation).score(rfm_segmentation) for i in range(len(kmeans))]
plt.plot(Nc,score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [ ]:
#fitting data in Kmeans theorem.
kmeans = KMeans(n_clusters=3, random_state=0).fit(rfm_segmentation)

In [ ]:
# this creates a new column called cluster which has cluster number for each row respectively.
rfm_segmentation['cluster'] = kmeans.labels_

In [ ]:
#check our hypothesis

rfm_segmentation[rfm_segmentation.cluster == 0].head(10)

In [ ]:
'''
cluster 0 have high recency rate which is bad. cluster 1 and cluster 2 having low so they are in race of platinum
and gold customer.
'''
sns.boxplot(rfm_segmentation.cluster,rfm_segmentation.Recency)

In [ ]:
'''
cluster 0 have low frequency rate which is bad. cluster 1 and cluster 2 having high so they are in 
race of platinum and gold customer.
'''
sns.boxplot(rfm_segmentation.cluster,rfm_segmentation.Frequency)

In [ ]:
''''
cluster 0 have low Monetary rate which is bad. cluster 1 have highest Montary (money spend) platinum where as 
cluster 2 have medium level(Gold) and cluster 0 is silver customer.
'''

sns.boxplot(rfm_segmentation.cluster,rfm_segmentation.Monetary)

Based on customer Segmentation we found out cluster 1 is Platinum customers Cluster 2 is Gold Customers Cluster 3 is Silver Customers

RFM Analysis

RFM (Recency, Frequency, Monetary) analysis is a customer segmentation technique that uses past purchase behavior to divide customers into groups. RFM helps divide customers into various categories or clusters to identify customers who are more likely to respond to promotions and also for future personalization services.

1. RECENCY (R): Days since last purchase
1. FREQUENCY (F): Total number of purchases
1. MONETARY VALUE (M): Total money this customer spent.

We will create those 3 customer attributes for each customer.